In [7]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [4]:
### NON SERVE PIU' (per ora lo tengo)
#Importo data di tweet e retweets
tweets=pd.read_csv("tweets.csv")
tweets=tweets.drop("Unnamed: 0",axis=1)
retweetted=pd.read_csv("retweetted.csv")
retweetted=retweetted.drop("Unnamed: 0",axis=1)
#lettura dei tweet più retweettati
pd.set_option('display.max_colwidth', None)
#selezione dei tweet più retwettati 
top20_rt=retweetted.sort_values("n_rt",ascending=False).drop_duplicates().head(20)
#selezioniamo gli id dei tweet nella top20_rt raggruppandoli in novax,neutro e provax
id_novax=[1472893531769430019,1469030399368118273,1473498816510889985,1472641059461152768,1473210101557194755,
            1474741256626524169,1465964667197661184,1464198361708478466,1472706459284787202,1472857869053747202]
id_critica=[1434976371336433667,1469030399368118273,1471088440259362818,1472301392467476489,1472859105866006530,
            1461948149845606401,1469340420064825346,1473031460584046601]
id_provax=[1472712138515001357,1473682381831917578]
#assegnazione cluster novax
for codice in id_novax:
    top20_rt.loc[top20_rt['id'] ==codice, 'cluster'] = "novax"
#assegnazione cluster neutro
for codice in id_critica:
    top20_rt.loc[top20_rt['id'] ==codice, 'cluster'] = "critica"
#assegnazione cluster provax
for codice in id_provax:
    top20_rt.loc[top20_rt['id'] ==codice, 'cluster'] = "provax"
#top20_rt["cluster"]

FileNotFoundError: [Errno 2] No such file or directory: 'tweets.csv'

##### Importazione e trasformazione dati (presa da tweets.ipynb)

In [53]:
tweets_2=pd.read_csv("tweets2_nogreenpass_vaccinatevi_casapound.csv")
retweetted=pd.read_csv("retweetted2_nogreenpass_vaccinatevi_casapound.csv")

In [54]:
rt_tweets=tweets_2[tweets_2.retweet==True][['user_id','user_id_retweet','author','colours']]

In [55]:
tweets_ids=rt_tweets[['user_id','author','colours']]
retwitted_ids=retweetted[['author_id','author','colours']].rename(columns={'author_id':'user_id'})
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).drop_duplicates(['user_id','author']).reset_index(drop=True)

##### Creazione rete con NetworkX

In [61]:
user_ret_network= nx.DiGraph()
for index,row in rt_tweets.iterrows():
    user_ret_network.add_edge(row['user_id'],row['user_id_retweet']) 
# manca il peso, si potrebbe usare il numero di volte che un utente retwetta un altro utente

In [80]:
adj=nx.adjacency_matrix(user_ret_network)

##### Clustering 
Kmeans

In [66]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(adj)

AffinityPropagation

In [79]:
from sklearn.cluster import AffinityPropagation
APropagation= AffinityPropagation(random_state=5).fit(adj)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_affinity_propagation.py:243: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn("Affinity propagation did not converge, this model "


In [68]:
#ho ricreato nodes e edges perchè non ricordavo quali file fossero
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.user_id,'Label':rt_tweets_ids.author,'Interval':None,'Colour':rt_tweets_ids.colours})
Edges_df=pd.DataFrame({'Source':rt_tweets.user_id,'Target':rt_tweets.user_id_retweet,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':1.0})

In [72]:
# Aggiunta del cluster come colonna di nodes
Nodes_df["Kmeans"]=kmeans.labels_
Nodes_df.head(20)

,Id,Label,Interval,Colour,Kmeans
0,354747398,MarcG_69,None,#fb1239,0
1,885303356,rekotc,None,#fb1239,0
2,19592755,xposeidone,None,#fb1239,2
3,494358540,the_bullet_xxx,None,#fb1239,0
4,1225034977,GiovanniTarchi,None,#fb1239,2
5,1219944530,Samira1577,None,#fb1239,0
6,492397709,bordienrico,None,#fb1239,0
7,603116709,CosimoSoulGuide,None,#fb1239,0
8,1215290216,bushido01,None,#fb1239,0
9,1241292304004022272,Glo96615250,None,#aaaacc,2
